In [3]:
import ast
import string

In [4]:
inputLines = open("puzzle18.txt").read().split("\n")
numbers = [ ast.literal_eval(x) for x in inputLines ]

In [5]:
def add(a, b):
    return [a, b]
def split(a):
    l = a // 2
    r = a // 2 + a % 2
    return [l, r]

In [6]:
def adds(a, b):
    return f"[{a},{b}]"
def splits(a):
    n = int(a)
    l = n // 2
    r = n // 2 + n % 2
    return f"[{l},{r}]"

In [7]:
nested = "[[[[[5,6], 3]]]]"

# Explosion testing

In [21]:
# nested = "[[[[[9,8],1],2],3],4]" # right only
# nested = "[7,[6,[5,[4,[3,2]]]]]" # left only
nested = "[[6,[5,[4,[3,2]]]],1]" # both ways
nested = "[[3,[2,[1,[7,3]]]],[7,[5,[4,[3,2]]]]]" # [3,2] unaffected
cursor = 0
depth = 0
expReq = False
expDone = False
leftNum, rightNum = -1, -1
print(nested, len(nested))
while cursor < len(nested):
    c = nested[cursor]
    if c == "[": depth += 1
    elif c == "]": depth -= 1

    # print(nested)
    # print(" " * cursor + "^", cursor, depth)

    if depth == 5:
        firstClosingBracket = nested[cursor:].find("]") + cursor
        print("EXPLODING:")
        print(nested)
        print(" " * cursor + "^" + " " * (firstClosingBracket - cursor - 1) + "^")

        pair = nested[cursor + 1:firstClosingBracket].split(",")
        l, r = int(pair[0]), int(pair[1])
        lcursor,rcursor = cursor,firstClosingBracket
        # Find first number at the left
        while lcursor > 1:
            if nested[lcursor] in string.digits: break
            lcursor -= 1
        # Find first number at the right
        while rcursor < len(nested) - 1:
            if nested[rcursor] in string.digits: break
            rcursor += 1
        # print(nested)
        # print(" " * cursor + "^ < cursor")
        # print(" " * firstClosingBracket + "^ < closing bracket")
        # print(" " * lcursor + "^ < lcurs")
        # print(" " * rcursor + "^ < rcurs")
        ln = ""
        lstart = lcursor
        if lcursor > 1:
            if nested[lcursor -1] in string.digits:
                lstart = lcursor - 1
            ln = str(int(nested[lstart: lcursor+1]) + l)
        rn = ""
        rend = rcursor
        if rcursor < len(nested) - 1:
            if nested[rcursor+1] in string.digits:
                rend = rcursor + 1
            rn = str(int(nested[rcursor:rend + 1]) + r)
        
        if ln != "":
            nested = nested[:lstart] + ln + nested[lcursor + 1:]
            rcursor += len(ln) - 1
            cursor += len(ln) - 1
            firstClosingBracket += len(ln) - 1
        if rn != "":
            nested = nested[:rcursor] + rn + nested[rend + 1:]
    
        nested = nested[:cursor] + "0" + nested[firstClosingBracket + 1:]
        cursor = firstClosingBracket
        print("EXPLODED")
        print(nested)
        break
    cursor+=1

[[3,[2,[1,[7,3]]]],[7,[5,[4,[3,2]]]]] 37
EXPLODING:
[[3,[2,[1,[7,3]]]],[7,[5,[4,[3,2]]]]]
          ^   ^
EXPLODED
[[3,[2,[8,0]]],[10,[5,[4,[3,2]]]]]


# Explosion implementation

In [59]:
def explodes(a):
    nested = a
    cursor = 0
    depth = 0
    expReq = False
    expDone = False
    leftNum, rightNum = -1, -1
    while cursor < len(nested):
        c = nested[cursor]
        if c == "[": depth += 1
        elif c == "]": depth -= 1

        if depth == 5:
            firstClosingBracket = nested[cursor:].find("]") + cursor
            pair = nested[cursor + 1:firstClosingBracket].split(",")

            l, r = int(pair[0]), int(pair[1])
            lcursor,rcursor = cursor,firstClosingBracket

            # Find first number at the left
            while lcursor > 0:
                if nested[lcursor] in string.digits: break
                lcursor -= 1

            # Find first number at the right
            while rcursor < len(nested):
                if nested[rcursor] in string.digits: break
                rcursor += 1

            ln = ""
            lstart = lcursor
            if lcursor > 1:
                if nested[lcursor -1] in string.digits:
                    lstart = lcursor - 1
                ln = str(int(nested[lstart: lcursor+1]) + l)

            rn = ""
            rend = rcursor
            if rcursor < len(nested) - 1:
                if nested[rcursor+1] in string.digits:
                    rend = rcursor + 1
                rn = str(int(nested[rcursor:rend + 1]) + r)
            
            if ln != "":
                nested = nested[:lstart] + ln + nested[lcursor + 1:]
                loffset = len(ln) - 1
                rcursor += loffset
                rend += loffset
                cursor += loffset
                firstClosingBracket += loffset
            if rn != "":
                nested = nested[:rcursor] + rn + nested[rend + 1:]
                roffset = len(rn) - 1
                firstClosingBracket += roffset
        
            nested = nested[:cursor] + "0" + nested[firstClosingBracket + 1:]
            cursor = firstClosingBracket
            # print("EXPLODED")
            # print(nested)
            return nested, True
        cursor+=1
    return nested, False


In [60]:
a = explodes("[[[[[9,8],1],2],3],4]") # right only
b = explodes("[7,[6,[5,[4,[3,2]]]]]") # left only
c = explodes("[[6,[5,[4,[3,2]]]],1]") # both ways
d = explodes("[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]") # [3,2] unaffected
e = explodes("[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]")
a[0], b[0], c[0], d[0], e[0]

('[[[[0,9],2],3],4]',
 '[7,[6,[5,[7,0]]]]',
 '[[6,[5,[7,0]]],3]',
 '[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]',
 '[[3,[2,[8,0]]],[9,[5,[7,0]]]]')

# Splitting test

In [45]:
tosplit = "[11, 2]"
cursor = 0
while cursor < len(tosplit):
    if tosplit[cursor] in string.digits and tosplit[cursor + 1] in string.digits:
        n = int(tosplit[cursor: cursor + 2])
        l,r = n // 2, n // 2 + n % 2
        tosplit = tosplit[:cursor] + str([l, r]) + tosplit[cursor + 2:]
        print(tosplit)
        break
    cursor += 1

[[5, 6], 2]


# Splitting implementation

In [46]:
def splits(a):
    tosplit = a
    cursor = 0
    while cursor < len(tosplit):
        if tosplit[cursor] in string.digits and tosplit[cursor + 1] in string.digits:
            n = int(tosplit[cursor: cursor + 2])
            l,r = n // 2, n // 2 + n % 2
            tosplit = tosplit[:cursor] + str([l, r]).replace(" ", "") + tosplit[cursor + 2:]
            return tosplit, True
        cursor += 1
    return tosplit, False

In [47]:
splits("[11, 2]")

('[[5,6], 2]', True)

In [48]:
def reduces(a):
    print("Reducing:", a)
    while True:
        a, r = explodes(a)
        if r: print("Exploded:", a)
        else:
            a, r = splits(a)
            if r: print("Reduced:", a)
            else: break
    return a


In [53]:
x, y = "[[[[4,3],4],4],[7,[[8,4],9]]]", "[1,1]"
z = adds(x, y)
res = "[[[[0,7],4],[[7,8],[6,0]]],[8,1]]".replace(" ", "")

In [54]:
reduces(z), res == reduces(z)

Reducing: [[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]
Exploded: [[[[0,7],4],[7,[[8,4],9]]],[1,1]]
Exploded: [[[[0,7],4],[15,[013]]],[1,1]]
Reduced: [[[[0,7],4],[[7,8],[013]]],[1,1]]
Reduced: [[[[0,7],4],[[7,8],[[0,1]3]]],[1,1]]
Exploded: [[[[0,7],4],[[7,8],[04]]],[1,1]]
Reduced: [[[[0,7],4],[[7,8],[[2,2]]]],[1,1]]
Exploded: [[[[0,7],4],[[7,10],[0]]],[3,1]]
Reduced: [[[[0,7],4],[[7,[5,5]],[0]]],[3,1]]
Exploded: [[[[0,7],4],[[12,0],[5]]],[3,1]]
Reduced: [[[[0,7],4],[[[6,6],0],[5]]],[3,1]]
Exploded: [[[[0,7],10],[[0,6],[5]]],[3,1]]
Reduced: [[[[0,7],[5,5]],[[0,6],[5]]],[3,1]]
Reducing: [[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]
Exploded: [[[[0,7],4],[7,[[8,4],9]]],[1,1]]
Exploded: [[[[0,7],4],[15,[013]]],[1,1]]
Reduced: [[[[0,7],4],[[7,8],[013]]],[1,1]]
Reduced: [[[[0,7],4],[[7,8],[[0,1]3]]],[1,1]]
Exploded: [[[[0,7],4],[[7,8],[04]]],[1,1]]
Reduced: [[[[0,7],4],[[7,8],[[2,2]]]],[1,1]]
Exploded: [[[[0,7],4],[[7,10],[0]]],[3,1]]
Reduced: [[[[0,7],4],[[7,[5,5]],[0]]],[3,1]]
Exploded: [[[[0,7],4],[[12,0

('[[[[0,7],[5,5]],[[0,6],[5]]],[3,1]]', False)

In [152]:
[[9,5], [0,1]]

False

In [61]:
nums = open("puzzle18_es.txt").read().split("\n")
r = nums[0]
for x in range(1, len(nums)):
    r = adds(r, nums[x])
    r = reduces(r)
print(r)

Reducing: [[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]],[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]]
Exploded: [[[[4,0],[5,0]],[[[4,5],[2,6]],[9,5]]],[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]]
Exploded: [[[[4,0],[5,4]],[[0,[7,6]],[9,5]]],[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]]
Exploded: [[[[4,0],[5,4]],[[7,0,[15,5]]],[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]]
Exploded: [[[[4,0],[5,4]],[[7,15,0],[12,[[[3,7],[4,3]],[[6,3],[8,8]]]]]


ValueError: invalid literal for int() with base 10: '[[3'